# Day 6: Resonant Collinearity

## Import libraries

In [6]:
import copy

## Import data

In [10]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - The grid map represents antennas.
# - EACH antenna is represented by a *letter* or *digit*.
# - The FREQUENCY of EACH antenna is different, based on whether the antenna is a: lowercase letter, uppercase letter or digit.
# - '#': Represents an antinode. 
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-8_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

# Separate data by line to create rows for grid
grid = file_data.split("\n")

# Separate data in EACH row to represent EACH column
for i in range(len(grid)):
    grid[i] = list(grid[i])

print(grid)
# ====================================================================================================================

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'], ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'], ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]


## Helper functions

In [11]:
def find_twice_as_far_pairs(grid):
    # Parse the grid into a 2D list
    # grid = [list(row) for row in grid]
    n_rows, n_cols = len(grid), len(grid[0])
    
    # Store positions of each letter
    letter_positions = {}
    for i in range(n_rows):
        for j in range(n_cols):
            char = grid[i][j]
            if char.isalpha():  # Only consider letters
                if char not in letter_positions:
                    letter_positions[char] = []
                letter_positions[char].append((i, j))
    
    # Find pairs that are twice as far
    twice_as_far_pairs = []
    for letter, positions in letter_positions.items():
        n_positions = len(positions)
        for i in range(n_positions):
            for j in range(i + 1, n_positions):
                # Calculate Manhattan distance
                dist1 = abs(positions[i][0] - positions[j][0]) + abs(positions[i][1] - positions[j][1])
                for k in range(n_positions):
                    if k != i and k != j:
                        # Compare with other distances
                        dist2 = abs(positions[i][0] - positions[k][0]) + abs(positions[i][1] - positions[k][1])
                        if dist1 == 2 * dist2 or dist2 == 2 * dist1:
                            twice_as_far_pairs.append((letter, positions[i], positions[j]))
    
    return twice_as_far_pairs

## Part 1

In [16]:
# *** [PART 1] ***
# ! PROBLEM: The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas.
# - In particular, an *antinode* occurs at *any point* that is perfectly *in line* with TWO antennas of the SAME frequency - but ONLY when ONE of the antennas is TWICE as far away as the other. This means that for any PAIR of antennas with the SAME frequency, there are TWO antinodes, one on EITHER side of them.
# - Antennas with DIFFERENT frequencies DO NOT create antinodes; however, antinodes CAN occur at the SAME locations that CONTAIN antennas.
# - TODO: Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?
# ====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part1_grid = copy.deepcopy(grid)

result = find_twice_as_far_pairs(part1_grid)
print(result)

for pair in result:
    print(pair)
    #print(pair[0], "(", pair[1], pair[2], ")")
# ====================================================================================================================

[]


## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: xxx
# - TODO: xxx
#====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part2_grid = copy.deepcopy(grid)
